In [1]:
import pandas as pd
import pickle

In [2]:
dataset = pd.read_csv("50_Startups.csv")
dataset.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [3]:
dataset = pd.get_dummies(dataset,dtype=int, drop_first=True)
dataset.head()

,R&D Spend,Administration,Marketing Spend,Profit,State_Florida,State_New York
0,165349.20,136897.80,471784.10,192261.83,0,1
1,162597.70,151377.59,443898.53,191792.06,0,0
2,153441.51,101145.55,407934.54,191050.39,1,0
3,144372.41,118671.85,383199.62,182901.99,0,1
4,142107.34,91391.77,366168.42,166187.94,1,0


In [4]:
dataset.columns

Index(['R&D Spend', 'Administration', 'Marketing Spend', 'Profit',
       'State_Florida', 'State_New York'],
      dtype='object')

In [5]:
independent = dataset[['R&D Spend', 'Administration', 'Marketing Spend','State_Florida', 'State_New York']]
independent.head()

,R&D Spend,Administration,Marketing Spend,State_Florida,State_New York
0,165349.20,136897.80,471784.10,0,1
1,162597.70,151377.59,443898.53,0,0
2,153441.51,101145.55,407934.54,1,0
3,144372.41,118671.85,383199.62,0,1
4,142107.34,91391.77,366168.42,1,0


In [6]:
dependent = dataset[["Profit"]]
dependent.head()

,Profit
0,192261.83
1,191792.06
2,191050.39
3,182901.99
4,166187.94


In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(independent, dependent, test_size=0.30, random_state=0)

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

#Here the fit.transform function Standardize the Undefined data to Defined Data and make it to be saved in X_train
X_train = sc.fit_transform(X_train)

#Here the transform function uses the process occurred in fit.transform function for the input X_test and make it to be saved in X_train
X_test = sc.transform(X_test)

X_test

array([[-0.1403821 ,  2.28593993, -0.63280437,  2.        , -0.76870611],
       [ 0.5692117 , -1.24096039,  0.37552686, -0.5       , -0.76870611],
       [ 0.59465017, -0.51102691,  0.21780907,  2.        , -0.76870611],
       [-0.92249538, -1.51579286, -0.27786722,  2.        , -0.76870611],
       [ 1.65079661, -0.87781077,  1.58762665,  2.        , -0.76870611],
       [-0.01625436,  0.15939469,  1.16810991, -0.5       ,  1.30088727],
       [-1.07956593, -2.24414796, -0.11853428, -0.5       ,  1.30088727],
       [-0.24112469,  1.1235571 , -0.86213689, -0.5       ,  1.30088727],
       [ 0.02242149, -0.03787582,  0.78603258,  2.        , -0.76870611],
       [ 1.41848712, -1.25644297,  1.26760194,  2.        , -0.76870611],
       [-0.3567741 , -0.80359112,  0.10650537,  2.        , -0.76870611],
       [-0.55107837, -1.50272366,  0.03664684, -0.5       ,  1.30088727],
       [ 0.04977002,  0.79103569, -0.5109594 ,  2.        , -0.76870611],
       [-0.54263284,  1.31734166,  0.0

In [9]:
from sklearn.svm import SVR
regressor = SVR (kernel = 'linear', C = 10000)
regressor.fit(X_train,Y_train)

C:\Anaconda\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=10000, kernel='linear')

In [10]:
# To know How many Support Vectors has been taken by SVM
regressor.n_support_

array([35])

In [11]:
# To make it in order of Support Vectors those been taken by SVM
regressor.support_

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34])

In [12]:
#prediction
#Keyword - predict is function of Application 
Y_pred=regressor.predict(X_test)

In [13]:
# Evaluation Metrics - Calculate R-Square Value
from sklearn.metrics import r2_score
r_score = r2_score(Y_test,Y_pred)
r_score

0.9239983428118113

In [14]:
# Naming the Model with its extension .sav
filename = "finalized_model_SVM_Regression.sav"

#Model will be saved in respective path, once after the below Code Execution
# Keyword - wb refers to "write binary"
saved_model = pickle.dump(regressor,open(filename,'wb'))

In [15]:
# Preprocessing the input before Providing it to Deployment Model (Finalized Model)
pre_input = sc.transform([[10000,20000,3000,1,0]])
pre_input

C:\Anaconda\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[-1.28923552, -4.02780167, -1.51510487,  2.        , -0.76870611]])

In [16]:
# Load the Saved Model and check the same to send it for Phase-02 (Deployment)
# # Keyword - rb refers to "read binary"
finalized_model=pickle.load(open(filename,'rb'))
finalized_model_result = finalized_model.predict(pre_input) 
finalized_model_result

array([45980.71967589])